In [ ]:
# as png
# %matplotlib inline 
#  interactable inside ide
# %matplotlib widget
### interactable seperate window
%matplotlib tk 

import mne
import numpy as np
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')
from scipy.io import loadmat

In [ ]:
# def extractBCI3(person_id = [0]):
person_id = [0]
files = ['/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_aa_mat/100Hz/data_set_IVa_aa.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_al_mat/100Hz/data_set_IVa_al.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_av_mat/100Hz/data_set_IVa_av.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_aw_mat/100Hz/data_set_IVa_aw.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_ay_mat/100Hz/data_set_IVa_ay.mat',]

dat = loadmat(files[1], struct_as_record=True)
extra_ch =['AFp1', 'AFp2', 'FAF5', 'FAF1', 'FAF2', 'FAF6', 'FFC7', 'FFC5', 'FFC3', 'FFC1', 'FFC2', 'FFC4', 'FFC6', 
    'FFC8', 'CFC7', 'CFC5', 'CFC3', 'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'CCP7', 'CCP5', 'CCP3', 'CCP1', 'CCP2',
    'CCP4', 'CCP6', 'CCP8', 'PCP7', 'PCP5', 'PCP3', 'PCP1', 'PCP2', 'PCP4', 'PCP6', 'PCP8', 'PPO7', 'PPO5', 'PPO1',
    'PPO2', 'PPO6', 'PPO8', 'OPO1', 'OPO2', 'OI1', 'OI2', 'I1', 'I2']

In [ ]:
sfreq = dat['nfo']['fs'][0][0][0][0]
EEGdata   = dat['cnt'].T
EEGdata.astype('float64')
EEGdata = EEGdata*0.1
nchannels, nsamples = EEGdata.shape

chan_names = [s[0] for s in dat['nfo']['clab'][0][0][0]]

event_onsets  = dat['mrk'][0][0][0][0]
event_codes   = dat['mrk'][0][0][1][0]
# event_codes = np.nan_to_num(event_codes)

labels = np.zeros((1, nsamples), int)
labels[0, event_onsets] = event_codes

cl_lab = [s[0] for s in dat['mrk']['className'][0][0][0]]
cl_lab.append('nan')
cl1    = cl_lab[0]
cl2    = cl_lab[1]

# digitized electrode positions 
xpos = dat['nfo']['xpos']
ypos = dat['nfo']['ypos']

nclasses = len(cl_lab)
nevents = len(event_onsets)

# Print some information
print('Shape of EEG:', EEGdata.shape)
print('Sample rate:', sfreq)
print('Number of channels:', nchannels)
print('Channel names:', chan_names)
print('Number of events:', len(event_onsets))
print('Event codes:', np.unique(event_codes))
print('Class labels:', cl_lab)
print('Number of classes:', nclasses)

In [ ]:
chan_names

In [ ]:
# Dictionary to store the trials in, each class gets an entry
trials = {}

# The time window to extract for each trial, here 0. -- 3.5 seconds
win = np.arange(int(0*sfreq), int(3.5*sfreq))

# Length of the time window
nsamples = len(win)

# Loop over the classes (right, foot)
for cl, code in zip(cl_lab, np.unique(event_codes)):
    
    # Extract the onsets for the class
    cl_onsets = event_onsets[event_codes.astype('str') == str(code)]
    
    # Allocate memory for the trials
    trials[cl] = np.zeros((nchannels, nsamples, len(cl_onsets)))
    
    # Extract each trial
    for i, onset in enumerate(cl_onsets):
        trials[cl][:,:,i] = EEGdata[:, win+onset]
        
# the dimensionality of the data (channels x time x trials)
print('Shape of trials[cl1]:', trials[cl1].shape)
print('Shape of trials[cl2]:', trials[cl2].shape)
# print('Shape of trials[00]:', trials[0].shape)

In [ ]:
right_hand  = np.rollaxis(trials[cl1], 2, 0)  
foot = np.rollaxis(trials[cl2], 2, 0) 
test = np.rollaxis(trials['nan'], 2, 0) 
data = np.concatenate([right_hand, foot, test])

In [ ]:
Y = np.concatenate([-np.ones(right_hand.shape[0]),
                     np.ones(foot.shape[0]),
                     np.zeros(test.shape[0])])
data.shape

In [ ]:
# Initialize an info structure
info = mne.create_info(
        ch_names = chan_names,
        ch_types = ['eeg']*nchannels,
        sfreq    = sfreq )  
# info.set_montage('standard_1020')
print('Event created :', info)


# Electrode Locations
xpos = dat['nfo']['xpos'][0][0]
ypos = dat['nfo']['ypos'][0][0]
layout_pos = np.concatenate([xpos, ypos], axis = 1)
layout = mne.channels.generate_2d_layout(
    xy = layout_pos,
    ch_names=chan_names,
    name ='EEG custom layout',
    )

In [ ]:
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
new_name = []
for ch in ten_twenty_montage.ch_names:
    ch = ch.lower()
    new_name.append(ch[0].upper() + ch[1:])

ten_twenty_montage.ch_names = new_name

ch_names = info['ch_names']
new_name = []
for ch in ch_names:
    ch = ch.lower()
    new_name.append(ch[0].upper() + ch[1:])

# # Remove dots from channel names in raw.info['ch_names']
# new_names = []
# for ch_name in new_name:
#     new_names.append(ch_name.split('.')[0])

mne.rename_channels(info, dict(zip(info['ch_names'], new_name)))
# info.set_montage(ten_twenty_montage, verbose=False, on_missing='ignore');
# print(ten_twenty_montage.ch_names)
# print(chan_names)

In [70]:
raw = mne.io.RawArray(EEGdata, info)
# raw = raw.pick_channels(ch_names = ten_twenty_montage.ch_names).copy()
# mon = mne.channels.read_dig_fif('Physionet_Chloc.fif')
# raw.set_montage('standard_1020')
# raw.set_montage(mon, on_missing='warn')
scale = dict(mag=1e-12, grad=4e-11, eeg=100e-6)
rawfltrd = raw.filter(1, 30, verbose= False, fir_design='firwin', skip_by_annotation='edge').copy()

Creating RawArray with float64 data, n_channels=118, n_times=283574
    Range : 0 ... 283573 =      0.000 ...  2835.730 secs
Ready.


In [ ]:
mon = mne.channels.read_dig_fif('Physionet_Chloc.fif')
# mon.rename_channels(dict(zip(mon.ch_names,raw.info['ch_names'])))
# mne.rename_channels(info, dict(zip(info['ch_names'], new_name)))
# raw.set_montage('Physionet_Chloc.fif')
# raw.set_montage(mon, on_missing='warn');

In [ ]:
locinfo = mne.io.read_raw_fif('Physionet_ChLoc_raw.fif', preload = False)
locinfo.pick_channels(raw.ch_names)
locinfo.ch_names

In [76]:
raw._set_channel_positions(locinfo._get_channel_positions(), locinfo.ch_names)

In [77]:
raw.plot_sensors(show_names = True);

### Artifact Removal

In [ ]:
rawfltrd_car = rawfltrd.copy()
rawfltrd_car.set_eeg_reference(ref_channels='average', projection=True);

In [ ]:
rawfltrd_ssp = rawfltrd.copy()
eog_proj, events = mne.preprocessing.compute_proj_eog(rawfltrd_ssp, n_grad=0, n_mag=0, n_eeg=2, average=True, verbose=False, ch_name = ['Fpz','Fp1','Fp2'], reject=None) # returns EOG Proj and events of blinks
rawfltrd_ssp.add_proj(projs=eog_proj);

In [ ]:
# rawfltrd_ssp.plot(scalings = scale);

In [ ]:
# cov = mne.Covariance()
rawfltrd_ica = rawfltrd_car.apply_proj().copy()
ica = mne.preprocessing.ICA(n_components = 20, noise_cov= None, random_state=2, method='picard',max_iter=500)
# Create an instance of RAW
# rawfltrd_ica = raw.copy()
# rawfltrd_ica.apply_proj()
ica.fit(rawfltrd_ica);
# n_components  = 10 then ica.exclude = [1,2]
ica.exclude = []
# Using EOG Channel to select ICA Components
ica.exclude , ex_scores = ica.find_bads_eog(rawfltrd_ica, ch_name=['Fpz','Fp1','Fp2']);#,threshold=2);

In [ ]:
ica.exclude

In [ ]:
ica.plot_sources(rawfltrd_ica);
ica.plot_scores(ex_scores);

In [ ]:
rawfltrd_ica = ica.apply(rawfltrd_ica)

In [ ]:
rawfltrd.plot_psd();

### Epoching

In [ ]:
event_ids = dict(right = -1, foot = 1, test = 0)
eventLength = Y.shape[0]
ev = dat['mrk']['pos'][0][0][0] #[i*sfreq*3 for i in range(eventLength)]

event_marker = np.column_stack((np.array(ev,  dtype = int),
                          np.zeros(eventLength,  dtype = int),
                          np.array(Y,  dtype = int)))

ann = mne.annotations_from_events(event_marker, sfreq, event_desc = Y)

In [ ]:
rawfltrd.set_annotations(ann)
event_data = mne.events_from_annotations(rawfltrd)
event_marker, event_ids = event_data

In [ ]:
rawepochs = rawfltrd_ica.apply_proj().copy()
epochs = mne.Epochs(rawepochs, event_marker, event_ids, tmax = 1.0, tmin = -0.5)
# epochs = mne.EpochsArray(data, info, events,tmin=0.0, event_id= event_id)
epochs.apply_baseline()

In [ ]:
T1 = epochs['right']
T2 = epochs['foot']
t1 = epochs['right'].average()
t2 = epochs['foot'].average()
picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']

In [ ]:
clas = 'right'
# epochs['right'].plot_image(combine = 'mean', title = 'right');
epochs[clas].average().plot_image(show_names = 'all', titles = clas, picks = picks);
epochs[clas].plot_topo_image(layout=layout, title = clas);
epochs[clas].plot_image(title = clas, combine = 'mean');
epochs[clas].plot_psd();


In [ ]:
clas = 'right'
epochs[clas].average().plot_image(show_names = 'all',titles = clas, picks = picks);

### Feature Extraction

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

#### Power

In [ ]:
from mne.time_frequency import tfr_morlet

In [ ]:
frequencies =  np.arange(5,30,1) #np.logspace(*np.log10([5, 30]), num=25)
chpick = [1] #baseline = (-0.5, 0.),

In [ ]:
show_names = 'all'
    # power.plot(picks=[10],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
power.plot_joint(picks = ['C1','C3','C5'], timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = f'{list(T1.event_id.keys())[0]}')

In [ ]:
clas = 'foot'
power = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
for pick in chpick:
    # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
    power.plot_joint(mode = 'mean', title = clas)

In [ ]:
power = tfr_morlet(T2, freqs = frequencies, n_cycles = frequencies/2, return_itc= False, average = False)
# avg = power.average()
# power.plot_topomap();
# power.plot(picks = ['C2','C4','C6'], mode = 'mean');

In [ ]:
clas = 'right'
baset = [epochs.baseline[1] , -0.2]
power = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
powerDB = np.zeros(power.data.shape)
t = np.arange(np.abs(baset[1]-baset[0])*sfreq, dtype = np.int64)
for ch in range(power.data.shape[0]):
    for f in range(power.data.shape[1]):
        baseline = np.mean(power.data[ch,f,t])
        activity = power.data[ch,f,:]
        powerDB[ch,f,:] = 10*np.log10(activity/baseline)

newPower = mne.time_frequency.AverageTFR(epochs.info, powerDB,epochs.times, frequencies,1).copy()
newPower.plot_joint(title = clas);

In [ ]:
newPower.pick_channels(picks).plot_joint(title = clas);

In [ ]:
t1.plot_image(picks = ['C1','C3','C5','C2','C4','C6'],show_names = 'all');

In [ ]:
T1.plot_image(combine = 'mean');

In [ ]:
power.plot_topomap();

#### WST

In [ ]:
from kymatio.sklearn import Scattering2D
from kymatio.sklearn import Scattering1D
from kymatio.scattering2d.filter_bank import filter_bank
# from numpy.fft import fft
import matplotlib.pyplot as plt
# import numpy as np
# %matplotlib tk 

In [ ]:
M,N = epochs.get_data().shape[1:]
J,L = 3,8
scat2D = Scattering2D(J=J,shape=(M,N),L=L)
data = epochs['right','foot'].get_data()
data = data.reshape(224,-1)

In [ ]:
# Do this for every epochs 
Sx = scat2D(data)

### Classification

In [ ]:
train_data = epochs['right', 'foot'].get_data()
train_data = train_data.reshape(train_data.shape[0],-1)
labels = np.concatenate([-np.ones(right_hand.shape[0]),
                     np.ones(foot.shape[0])])

In [ ]:
print(train_data.shape)
print(labels.shape)

In [ ]:
clf = LDA()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
pipe = Pipeline([('scatter', scat2D), ('clf', clf)])
scores = cross_val_score(pipe, train_data, labels, cv = cv, verbose=False)
scores

In [ ]:
clf = SVC()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
pipe = Pipeline([('scatter', scat2D), ('clf', clf)])
scores = cross_val_score(pipe, train_data, labels, cv = cv, verbose=False)
scores

In [ ]:
clf = DTC()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
pipe = Pipeline([('scatter', scat2D), ('clf', clf)])
scores = cross_val_score(pipe, train_data, labels, cv = cv, verbose=False)
scores

In [ ]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
pipe.fit(epochs['right', 'foot'].get_data(), labels)
pipe.predict(epochs['test'][0].get_data())

In [ ]:
ans = []
for i in range(epochs['test'].get_data().shape[0]):
    ans.append(pipe.predict(epochs['test'][i].get_data()))

In [ ]:
ans

In [ ]:
epochs['right', 'foot'].get_data().shape

In [ ]:
labels.shape